#### RFM은 Recency, Frequency, Monetary 의 약자

- Recency : 최근성
- Frequency : 빈번함
- Monetary : 양 (현재 데이터에서는 결제한 총 금액)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(context='notebook',
              style='darkgrid',
              palette='pastel',
              font='Malgun Gothic',
              rc=custom_params)

In [3]:
sales_data = pd.read_csv("sales_05_8.csv")
sales_data.head()

,Install,Time_stamp,Status,ID,Payment,Game_Name
0,2022-01-14,2022-01-27,purchase,Heidi_635,19900,Soccer Manager
1,2022-03-10,2022-03-13,purchase,Courtney_067,900,Mine Ground
2,2022-02-13,2022-02-19,purchase,David_737,9900,Soccer Manager
3,2022-02-07,2022-02-25,purchase,Lindsay_820,4900,Mine Ground
4,2022-03-06,2022-03-10,purchase,Jon_823,49900,Soccer Manager


In [4]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75087 entries, 0 to 75086
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Install     75087 non-null  object
 1   Time_stamp  75087 non-null  object
 2   Status      75087 non-null  object
 3   ID          75087 non-null  object
 4   Payment     75087 non-null  int64 
 5   Game_Name   75087 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.4+ MB


In [5]:
sales_data["ID"].nunique()

52672

In [8]:
# datetime형 칼럼 생성 및, 기존 칼럼 datetime형으로 변경
sales_data["Now"] = pd.to_datetime("2022-06-10")
sales_data["Time_stamp"] = pd.to_datetime(sales_data["Time_stamp"])
sales_data = sales_data[sales_data["Status"] == "purchase"]
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27682 entries, 0 to 27681
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Install     27682 non-null  object        
 1   Time_stamp  27682 non-null  datetime64[ns]
 2   Status      27682 non-null  object        
 3   ID          27682 non-null  object        
 4   Payment     27682 non-null  int64         
 5   Game_Name   27682 non-null  object        
 6   Now         27682 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 1.7+ MB


In [10]:
sales_data["Interval"] = sales_data["Now"] - sales_data["Time_stamp"]
sales_data.head()

C:\Users\김영준\AppData\Local\Temp\ipykernel_9508\1677220807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data["Interval"] = sales_data["Now"] - sales_data["Time_stamp"]


,Install,Time_stamp,Status,ID,Payment,Game_Name,Now,Interval
0,2022-01-14,2022-01-27,purchase,Heidi_635,19900,Soccer Manager,2022-06-10,134 days
1,2022-03-10,2022-03-13,purchase,Courtney_067,900,Mine Ground,2022-06-10,89 days
2,2022-02-13,2022-02-19,purchase,David_737,9900,Soccer Manager,2022-06-10,111 days
3,2022-02-07,2022-02-25,purchase,Lindsay_820,4900,Mine Ground,2022-06-10,105 days
4,2022-03-06,2022-03-10,purchase,Jon_823,49900,Soccer Manager,2022-06-10,92 days


In [12]:
RFM_data = sales_data.groupby(["ID"]).agg({"Interval" : "min", "ID" : "count", "Payment" : "sum"})
RFM_data.head()

,Interval,ID,Payment
ID,,,
Aaron_027,60 days,5,40500
Aaron_031,102 days,5,17500
Aaron_077,40 days,4,26600
Aaron_165,21 days,4,49600
Aaron_224,108 days,1,900


In [14]:
RFM_data.columns = ["Recency", "Frequency", "Monetary"]
RFM_data.head()

,Recency,Frequency,Monetary
ID,,,
Aaron_027,60 days,5,40500
Aaron_031,102 days,5,17500
Aaron_077,40 days,4,26600
Aaron_165,21 days,4,49600
Aaron_224,108 days,1,900


In [15]:
RFM_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5267 entries, Aaron_027 to Zoe_638
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype          
---  ------     --------------  -----          
 0   Recency    5267 non-null   timedelta64[ns]
 1   Frequency  5267 non-null   int64          
 2   Monetary   5267 non-null   int64          
dtypes: int64(2), timedelta64[ns](1)
memory usage: 164.6+ KB


In [16]:
# 열에서 날짜 차이를 일(day) 단위로 추출하는 메서드인 dt.days
RFM_data["Recency"] = RFM_data["Recency"].dt.days
RFM_data

,Recency,Frequency,Monetary
ID,,,
Aaron_027,60,5,40500
Aaron_031,102,5,17500
Aaron_077,40,4,26600
Aaron_165,21,4,49600
Aaron_224,108,1,900
...,...,...,...
Zachary_937,101,4,11600
Zachary_948,48,5,26500
Zoe_163,11,5,105500


In [17]:
RFM_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5267 entries, Aaron_027 to Zoe_638
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Recency    5267 non-null   int64
 1   Frequency  5267 non-null   int64
 2   Monetary   5267 non-null   int64
dtypes: int64(3)
memory usage: 164.6+ KB


In [18]:
RFM_data.describe()
# 표준편차는 평균을 중심으로 각 요소 값들이 얼마나 떨어져 있나를 나타냄
# 표준편차가 75%값을 벗어나는 경우 데이터가 넓게 흩어져 있다는 의미가 됨
# 25%~75% 사분위수 범위

,Recency,Frequency,Monetary
count,5267.000000,5267.000000,5267.000000
mean,66.517752,5.255743,50340.003797
std,35.684775,6.995041,73012.364791
min,7.000000,1.000000,900.000000
25%,36.000000,1.000000,9900.000000
50%,61.000000,5.000000,29900.000000
75%,94.000000,5.000000,55500.000000
max,158.000000,37.000000,582700.000000
